In [0]:
dbutils.widgets.text("environment","devv","Enter the environment or catalog")
dbutils.widgets.text("once","yes","Do you want to run it once")
dbutils.widgets.text("process_time","5 seconds","Process time")

In [0]:
env = dbutils.widgets.get("environment")
once = True if dbutils.widgets.get("once") == "yes" else False
process_time = dbutils.widgets.get("process_time")
if once:
    print("starting sbit in Batch Mode")
else:
    print(f"starting sbit in Stream Mode with Processing Time = {process_time}")

In [0]:
spark.conf.set("spark.sql.shuffle.partitions", sc.defaultParallelism)
spark.conf.set("spark.databricks.delta.optimizeWrite.enabled", True)
spark.conf.set("spark.databricks.delta.autoCompact.enabled", True)
spark.conf.set("spark.sql.streaming.stateStore.providerClass", "com.databricks.sql.streaming.state.RocksDBStateStoreProvider")

In [0]:
%run ./02-setup

In [0]:
%run ./03-history-loader

In [0]:
sh = SetupHelper(env)
hl = HistoryLoader(env)

In [0]:
import pyspark.sql.functions as f
setup_check = spark.sql(f"SHOW DATABASES IN {sh.catalog}").filter(f.col('databaseName').isin(sh.silver_db,sh.bronze_db,sh.gold_db)).count() !=3
if setup_check:
    sh.setup()
    sh.validate()
    hl.history_loader()
    hl.validate()
else:
    print("data setup already present")


In [0]:
%run ./04-Bronze

In [0]:
bz = Bronze(env)

In [0]:
bz.load_bronze(once=once,process_time=process_time)

In [0]:
%run ./05-silver

In [0]:
sil = Silver(env)

In [0]:
sil.upsert(once=once,processing_time=process_time)

In [0]:
%run ./06-Gold

In [0]:
gol = Gold(env)

In [0]:
gol.upsert(once=once,process_time=process_time)